# Document Conversion - Quick start

## Getting started

The [Deep Search Toolkit](https://ds4sd.github.io/deepsearch-toolkit/) allows document conversion with the following few lines of code. It's that simple! For more info or step-by-step guide:
- Visit https://ds4sd.github.io/deepsearch-toolkit/guide/convert_doc/
- Follow this example notebook

### Set notebook parameters

In [22]:
from dsnotebooks.settings import ProjectNotebookSettings

# notebook settings auto-loaded from .env / env vars
notebook_settings = ProjectNotebookSettings()

#PROFILE_NAME = notebook_settings.profile  # the profile to use
#PROJ_KEY = notebook_settings.proj_key     # the project to use

PROFILE_NAME = "sds"
PROJ_KEY = "649f351a919a2be2acfd992605d19f8fb563e319"

# default project_key = 1234567890abcdefghijklmnopqrstvwyz123456



Project key:  649f351a919a2be2acfd992605d19f8fb563e319


### Import example dependencies

In [18]:
import deepsearch as ds
import json

import glob

from pathlib import Path
from zipfile import ZipFile

from deepsearch.documents.core.export import export_to_markdown
from IPython.display import display, Markdown, HTML, display_html

### Connect to Deep Search

In [19]:


api = ds.CpsApi.from_env(profile_name=PROFILE_NAME)

In [23]:
output_dir = Path("/Users/taa/Downloads/For_Peter_converted_docs_v4")

fnames = sorted(glob.glob("/Users/taa/Downloads/For_Peter/*.pdf"))

#for fname in fnames:
    #print(fname)
documents = ds.convert_documents(
        api=api,
        proj_key=PROJ_KEY,
        source_path="/Users/taa/Downloads/For_Peter/",
        progress_bar=True
)           
documents.download_all(result_dir=output_dir)
info = documents.generate_report(result_dir=output_dir)
print(info) 

Converting input:     : 100%|██████████████████████████████| 1/1 [00:57<00:00, 57.83s/it]                                                                                                                                                            


{'Total documents': 15, 'Successfully converted documents': 15}


In [24]:
# Iterare output files and visualize the output
for output_file in output_dir.rglob("json*.zip"):
    with ZipFile(output_file) as archive:
        all_files = archive.namelist()
        for name in all_files:
            if not name.endswith(".json"):
                print(f"skipping: ", name)
                continue
            
            basename = name.rstrip('.json')
            doc_json = json.loads(archive.read(f"{basename}.json"))
            doc_md = export_to_markdown(doc_json)

            ofile = output_dir / f"{basename}.md"
            with ofile.open("w") as fw:
                fw.write(doc_md)

            print(f"writing {ofile}")

skipping:  EPL_Soccer_Table.cells
writing /Users/taa/Downloads/For_Peter_converted_docs_v4/EPL_Soccer_Table.md
skipping:  IBM_Annual_Report_2022_Table.cells
writing /Users/taa/Downloads/For_Peter_converted_docs_v4/IBM_Annual_Report_2022_Table.md
skipping:  K231195_table_to_text.cells
writing /Users/taa/Downloads/For_Peter_converted_docs_v4/K231195_table_to_text.md
skipping:  Invoice.cells
writing /Users/taa/Downloads/For_Peter_converted_docs_v4/Invoice.md
skipping:  Services_Agreement_insurance_indemnification.cells


KeyError: "There is no item named 'Services_Agreement_insurance_indemnificati.json' in the archive"

In [23]:
# display last document
# display(Markdown(doc_md))

---

**There's more!**

The Deep Search Toolkit provides utility functions which can convert documents from different type of inputs.

- From a single url
- From a list of urls. In this case, the toolkit will launch a batch processing with all tasks.
- From a local PDF file
- From a local zip archive containing PDF files.
- From a local folder containing PDF files. In this case, the toolkit is packaging the files into batches and creates multiple zip archives.


---

## Let's explore document conversion

### Single URL

In [5]:
documents = ds.convert_documents(api=api, 
                                 proj_key=PROJ_KEY, 
                                 urls="https://arxiv.org/pdf/2206.00785.pdf", 
                                 progress_bar=True)

Converting input:     : 100%|██████████████████████████████| 1/1 [00:18<00:00, 18.78s/it]


In [6]:
# let's check what happened. 
# we generate a csv report about the conversion task and store it locally
result_dir = './converted_docs/'
info = documents.generate_report(result_dir=result_dir)
print(info)

{'Total documents': 1, 'Successfully converted documents': 1}


The saved report may help in debugging and analysing the conversion task

In [7]:
# let's download all the converted documents:
documents.download_all(result_dir=result_dir, progress_bar=True)

In [8]:
# the documents object stores some additional info like:
documents.statuses, documents.task_ids

(['SUCCESS'], ['ae6d4337-abf7-4c2f-8943-a9baf547b91a'])

### Multiple URLs

In [9]:
# let's create a list of urls we want to convert:
urls = ["https://arxiv.org/pdf/2206.00785.pdf", "https://arxiv.org/pdf/2206.01062.pdf"]

In [10]:
# Process multiple urls
documents = ds.convert_documents(
    api=api, 
    proj_key=PROJ_KEY, 
    urls= urls, 
    progress_bar=True
)

Converting input:     : 100%|██████████████████████████████| 2/2 [00:50<00:00, 25.42s/it]


In [11]:
# as before we can use the documents object to download all jsons. We can also iterate over them individually.
for doc in documents:
    doc.download(result_dir=result_dir, progress_bar=True)

### Process local file

In [12]:
documents = ds.convert_documents(
    api=api, 
    proj_key=PROJ_KEY, 
    source_path="../../data/samples/2206.01062.pdf", 
    progress_bar=True
)

Converting input:     : 100%|██████████████████████████████| 1/1 [00:33<00:00, 33.33s/it]


## Process folder of files

In [13]:
documents = ds.convert_documents(
    api=api, 
    proj_key=PROJ_KEY, 
    source_path="../../data/samples", 
    progress_bar=True
)

Converting input:     : 100%|██████████████████████████████| 1/1 [00:35<00:00, 35.31s/it]


In [14]:
info = documents.generate_report(result_dir)
print(info)

{'Total documents': 2, 'Successfully converted documents': 2}


In [15]:
# let's download all the converted documents:
documents.download_all(result_dir=result_dir,progress_bar=True)

---

## What's next?

Explore other examples which demonstrate possible use cases of the document conversion

- Visualize the text bounding boxes
- Extract figures
- Convert document to epub for your e-reader
